A simple analysis of this truss is performed by considering the force balance equations.  This may be expressed as a matrix which relates the unknown internal forces on all beams to the known externally applied forces (i.e. the truck).  The reaction forces pushing on the bridge are regarded as internal forces since they are unknown.  We write this relation as

$Af_i=−f_e$

Where $f_i$ is a vector (list) of the internal forces on each beam, and fe is the vector of known external forces. The matrix A holds the information detailing how the forces interact, i.e. the structure of the bridge.  The goal is to find the internal forces $f_i$.

![truss.png](truss.png)

## 1. Complete the above analysis and write down the entire matrix equation describing the forces at each joint.

### Left positive, right negative
### Up negative, down positive
### Positive - tension, Negative - compression
$$a_x \rightarrow -f_1cos(45)-f_2+r_{ax} = 0$$

$$a_y \rightarrow -f_1sin(45)+r_{ay} = 0$$

$$b_x \rightarrow f_1cos(45) - f_4 = 0$$

$$b_y \rightarrow f_1sin(45) + f_3 = 0 $$

$$c_x \rightarrow f_2 - f_5cos(45) - f_6 = 0$$

$$c_y \rightarrow -f_3 - f_5sin(45) = -T_c$$

$$d_x \rightarrow f_4 + f_5cos(45) - f_8cos(45) = 0$$

$$d_y \rightarrow f_5sin(45) + f_7 + f_8sin(45) = 0$$

$$e_x \rightarrow f_6 - f_9 = 0$$

$$e_y \rightarrow -f_7 = -T_e$$

$$f_x \rightarrow f_8 + f_9 = 0$$

$$f_y \rightarrow -f_8sin(45) + r_{fy} = 0$$

## 2. Write down the external force vector, $f_e$.  Recall that we take $T_c=T_e=$ truck weight / 2

$f_{ex} = 0$ there is no horizontal force

$$f_{ey} = \begin{pmatrix} 0 & 0 & T_c & 0 & T_e & 0 \end{pmatrix}$$

In [5]:
import math
import pandas as pd
from scipy import sparse

COLS = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'Rax', 'Ray', 'Rfy']

sin45 = math.sin(math.pi / 4)
cos45 = math.cos(math.pi / 4)

eqs = {
    'ax': {'f1': -cos45, 'f2': -1, 'Rax': 1},
    'ay': {'f1': -sin45, 'Ray': 1},
    'bx': {'f1': cos45, 'f4': -1},
    'by': {'f1': sin45, 'f3': 1},
    'cx': {'f2': 1, 'f5': -cos45, 'f6': -1},
    'cy': {'f3': -1, 'f5': -sin45},
    'dx': {'f4': 1, 'f5': cos45, 'f8': -cos45},
    'dy': {'f5': sin45, 'f7': 1, 'f8': sin45},
    'ex': {'f6': 1, 'f9': -1},
    'ey': {'f7': -1},
    'fx': {'f8': cos45, 'f9': 1},
    'fy': {'f8': -sin45, 'Rfy': 1}
}

rows = []
cols = []
data = []

for i, key in enumerate(eqs.keys()):
    for x, row in enumerate(eqs[key].keys()):
        rows.append(i)
        cols.append(COLS.index(row))
        data.append(eqs[key][row])

print("FULL MATRIX:")
mat = sparse.coo_matrix((data, (rows, cols)), shape=(len(eqs), len(COLS)))
mat_df = pd.DataFrame(mat.toarray(), columns=COLS)
mat_df['row'] = eqs.keys()
mat_df["ANS"] = [0 if a not in {'cy', 'ey'} else f'-T{a[0]}' for a in mat_df['row']]
mat_df = mat_df.set_index('row')
for col in mat_df.columns[:-1]:
    l = []
    for i, row in mat_df.iterrows():
        if abs(row[col]) == 1 or row[col] == 0:
            l.append(row[col])
            continue
        temp = "" if row[col] > 0 else "-"
        if "x" in row.name:
            temp += "cos(45)"
        elif "y" in row.name:
            temp += "sin(45)"
        l.append(temp)
    mat_df[col] = l
mat_df

FULL MATRIX:


,f1,f2,f3,f4,f5,f6,f7,f8,f9,Rax,Ray,Rfy,ANS
row,,,,,,,,,,,,,
ax,-cos(45),-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
ay,-sin(45),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
bx,cos(45),0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
by,sin(45),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
cx,0.0,1.0,0.0,0.0,-cos(45),-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
cy,0.0,0.0,-1.0,0.0,-sin(45),0.0,0.0,0.0,0.0,0.0,0.0,0.0,-Tc
dx,0.0,0.0,0.0,1.0,cos(45),0.0,0.0,-cos(45),0.0,0.0,0.0,0.0,0
dy,0.0,0.0,0.0,0.0,sin(45),0.0,1.0,sin(45),0.0,0.0,0.0,0.0,0
ex,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0


In [23]:
# Verify that equations are what we expect
print("Does our matrix yield the right equations?\n")
for _, row in mat_df.iterrows():
    s = []
    for col in mat_df.columns[:-1]:
        val = row[col]
        if val == 0:
            continue
        if val == 1:
            if not s:
                s.append(col)
                continue
            s.append(f'+ {col}')
            continue
        if val == -1:
            if not s:
                s.append(f'-{col}')
                continue
            s.append(f'- {col}')
            continue
        if val[0] == "-":
            if not s:
                s.append(f'-{col}{val[1:]}')
                continue
            s.append(f'- {col}{val[1:]}')
            continue
        if not s:
            s.append(f'{col}{val}')
            continue
        s.append(f'+ {col}{val}')
    print(row.name, '->', ' '.join(s), '=', row["ANS"], '\n')

Does our matrix yield the right equations?

ax -> -f1cos(45) - f2 + Rax = 0 

ay -> -f1sin(45) + Ray = 0 

bx -> f1cos(45) - f4 = 0 

by -> f1sin(45) + f3 = 0 

cx -> f2 - f5cos(45) - f6 = 0 

cy -> -f3 - f5sin(45) = -Tc 

dx -> f4 + f5cos(45) - f8cos(45) = 0 

dy -> f5sin(45) + f7 + f8sin(45) = 0 

ex -> f6 - f9 = 0 

ey -> -f7 = -Te 

fx -> f8cos(45) + f9 = 0 

fy -> -f8sin(45) + Rfy = 0 



## 3. Solve the system (using Matlab, or your preferred solver), and find the vector of member forces, $f_1$, $f_2$, $f_3$, etc.
## 4. Will the bridge collapse because of weight of the 15 ton truck?
## 5. What happens if somebody drives a 20 ton truck on this bridge?

In [31]:
# Sanity Checks
import numpy as np
from scipy.sparse.linalg import spsolve

existing_data = [COLS]
cols = []

MAX_COMPRESSION = 14
MAX_TENSION = 11
TRUCK_WEIGHTS = [15, 20]

for truck_weight in TRUCK_WEIGHTS:
    Tc = Te = truck_weight / 2

    # External Force Vector
    Fe = np.zeros(len(eqs))
    Fe[5] = -Tc
    Fe[9] = -Te
    ans = spsolve(mat, Fe)
    existing_data.append(ans)
    cols.append(f"Force at {truck_weight} tons")
    existing_data.append(["TENSION" if a > 0 else "COMPRESSION" if a < 0 else "" for a in ans])
    cols.append(f"{truck_weight}-ton t/c")
    will_break = ['' if -MAX_COMPRESSION < a < MAX_TENSION else "BREAK" for a in ans]
    if all(not a for a in will_break):
        will_break = ["GOOD"] * len(ans)
    existing_data.append(will_break)
    cols.append(f"Sturdy at {truck_weight}?")

df = pd.DataFrame(list(zip(*existing_data))).set_index(0)
df.columns = cols
df


/Users/anirudhkamath/opt/anaconda3/envs/MATH7203/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


,Force at 15 tons,15-ton t/c,Sturdy at 15?,Force at 20 tons,20-ton t/c,Sturdy at 20?
0,,,,,,
f1,-10.606602,COMPRESSION,GOOD,-14.142136,COMPRESSION,BREAK
f2,7.500000,TENSION,GOOD,10.000000,TENSION,
f3,7.500000,TENSION,GOOD,10.000000,TENSION,
f4,-7.500000,COMPRESSION,GOOD,-10.000000,COMPRESSION,
f5,-0.000000,,GOOD,-0.000000,,
f6,7.500000,TENSION,GOOD,10.000000,TENSION,
f7,7.500000,TENSION,GOOD,10.000000,TENSION,
f8,-10.606602,COMPRESSION,GOOD,-14.142136,COMPRESSION,BREAK
f9,7.500000,TENSION,GOOD,10.000000,TENSION,


In [27]:
# Some small sanity checks - I think I'm doing something wrong in these tests because everything else looks right, like the tension/compression stuff and the other test cases

vals = df.to_dict('index')

for truck_weight in TRUCK_WEIGHTS:
    tests = [
            (vals['Rfy'][f"Force at {truck_weight} tons"] + vals['Ray'][f"Force at {truck_weight} tons"], truck_weight),
            (vals['Rax'][f"Force at {truck_weight} tons"], 0),
            (vals['f1'][f"Force at {truck_weight} tons"], vals['f8'][f"Force at {truck_weight} tons"]),
            (vals['f2'][f"Force at {truck_weight} tons"], vals['f9'][f"Force at {truck_weight} tons"])
        ]
    for act, exp in tests:
        if act != exp:
            print("\n\n DIDN'T WORK\n", act, exp, "\n\n")
        else:
            print("\n\n WORKED\n", act, exp, "\n\n")




 DIDN'T WORK
 -15.000000000000002 15 




 WORKED
 0.0 0 




 WORKED
 -10.606601717798215 -10.606601717798215 




 WORKED
 7.500000000000002 7.500000000000002 




 DIDN'T WORK
 -20.0 20 




 WORKED
 0.0 0 




 WORKED
 -14.142135623730951 -14.142135623730951 




 WORKED
 10.000000000000002 10.000000000000002 


